In [4]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../py_files')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [6]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 5


# Check the drug

In [7]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [8]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-12 06:28:09] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 06:28:48] ax.service.ax_client: Generated new trial 80 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.0, 'LL_1': 0.0, 'LL_2': 1.0, 'LL_3': 0.0, 'P_1': 0.336583, 'P_2': 0.0, 'P_3': 1.0, 'S_1': 0.0, 'S_2': 0.61564, 'S_3': 1.0, 'Water': 1.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, axis=0, sort

11 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,87.500000,354.0,3.91,67.0,0.259750,0.009500,0.0,0.015625,0.029437,0.310937,0.0,0.063125,0.074375,0.237062,0.055688,0.336375,0.360938,0.247063
std,4.760952,0.0,0.00,0.0,0.058727,0.028468,0.0,0.062500,0.080637,0.063295,0.0,0.101469,0.108661,0.083951,0.082591,0.244909,0.135998,0.174087
min,80.000000,354.0,3.91,67.0,0.143000,0.000000,0.0,0.000000,0.000000,0.220000,0.0,0.000000,0.000000,0.103000,0.000000,0.094000,0.000000,0.000000
25%,83.750000,354.0,3.91,67.0,0.227500,0.000000,0.0,0.000000,0.000000,0.273250,0.0,0.000000,0.000000,0.165750,0.000000,0.197250,0.337750,0.000000
50%,87.500000,354.0,3.91,67.0,0.255500,0.000000,0.0,0.000000,0.000000,0.298500,0.0,0.000000,0.000000,0.266500,0.002500,0.239000,0.366000,0.344000
75%,91.250000,354.0,3.91,67.0,0.300000,0.000000,0.0,0.000000,0.000000,0.339250,0.0,0.114500,0.161500,0.297750,0.085250,0.416750,0.404000,0.367500
max,95.000000,354.0,3.91,67.0,0.359000,0.108000,0.0,0.250000,0.251000,0.484000,0.0,0.274000,0.313000,0.358000,0.261000,1.000000,0.568000,0.407000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.000000,16.000000,16.0,16.0000,16.000000,16.0,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,87.500000,0.354,0.391,0.067,0.842379,0.031677,0.0,0.0625,0.125000,1.0,0.0,0.216353,0.274112,0.774893,0.164807,0.631434,0.888550,0.687500
std,4.760952,0.000,0.000,0.000,0.155893,0.095161,0.0,0.2500,0.341565,0.0,0.0,0.362057,0.405117,0.275038,0.259583,0.178461,0.305594,0.478714
min,80.000000,0.354,0.391,0.067,0.571080,0.000000,0.0,0.0000,0.000000,1.0,0.0,0.000000,0.000000,0.379868,0.000000,0.292412,0.000000,0.000000
25%,83.750000,0.354,0.391,0.067,0.707717,0.000000,0.0,0.0000,0.000000,1.0,0.0,0.000000,0.000000,0.451254,0.000000,0.477890,1.000000,0.000000
50%,87.500000,0.354,0.391,0.067,0.860749,0.000000,0.0,0.0000,0.000000,1.0,0.0,0.000000,0.000000,1.000000,0.004002,0.664423,1.000000,1.000000
75%,91.250000,0.354,0.391,0.067,1.000000,0.000000,0.0,0.0000,0.000000,1.0,0.0,0.353549,0.565959,1.000000,0.245635,0.734295,1.000000,1.000000
max,95.000000,0.354,0.391,0.067,1.000000,0.361695,0.0,1.0000,1.000000,1.0,0.0,1.000000,1.000000,1.000000,0.808605,1.000000,1.000000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000000,16.0,16.00000,16.000000,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,87.500000,354.0,3.91,67.0,90.912500,3.325000,0.0,5.46875,10.303125,108.828125,0.0,22.093750,26.031250,82.971875,55.687500,336.375000,360.937500,247.062500,0.065625
std,4.760952,0.0,0.00,0.0,20.554468,9.963634,0.0,21.87500,28.223102,22.153081,0.0,35.514194,38.031302,29.382903,82.591137,244.908656,135.997779,174.087131,0.229288
min,80.000000,354.0,3.91,67.0,50.050000,0.000000,0.0,0.00000,0.000000,77.000000,0.0,0.000000,0.000000,36.050000,0.000000,94.000000,0.000000,0.000000,-0.350000
25%,83.750000,354.0,3.91,67.0,79.625000,0.000000,0.0,0.00000,0.000000,95.637500,0.0,0.000000,0.000000,58.012500,0.000000,197.250000,337.750000,0.000000,0.000000
50%,87.500000,354.0,3.91,67.0,89.425000,0.000000,0.0,0.00000,0.000000,104.475000,0.0,0.000000,0.000000,93.275000,2.500000,239.000000,366.000000,344.000000,0.000000
75%,91.250000,354.0,3.91,67.0,105.000000,0.000000,0.0,0.00000,0.000000,118.737500,0.0,40.075000,56.525000,104.212500,85.250000,416.750000,404.000000,367.500000,0.350000
max,95.000000,354.0,3.91,67.0,125.650000,37.800000,0.0,87.50000,87.850000,169.400000,0.0,95.900000,109.550000,125.300000,261.000000,1000.000000,568.000000,407.000000,0.350000


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.5
SL_1       0.1
SL_2       0.0
SL_3       0.1
LL_1       0.2
LL_2       1.7
LL_3       0.0
P_1        0.4
P_2        0.4
P_3        1.3
S_1        0.9
S_2        5.4
S_3        5.8
Water      4.0
Solvent    0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-12 06:41:07] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_5.json`.


# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [16]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [17]:
updated_script

'protocol/iteration_5_OT_2_protocol_.py'

# Perform experiment

# Results

In [18]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,80,354,3.91,67,0.300,0.000,0,0.00,0.000,0.300,0,0.101,0.000,0.300,0.000,0.235,0.382,0.382,5,0
1,81,354,3.91,67,0.359,0.000,0,0.00,0.000,0.359,0,0.022,0.000,0.259,0.000,0.195,0.403,0.403,5,0
2,82,354,3.91,67,0.233,0.000,0,0.00,0.000,0.319,0,0.000,0.313,0.135,0.005,0.431,0.564,0.000,6,0
3,83,354,3.91,67,0.300,0.044,0,0.00,0.000,0.300,0,0.000,0.055,0.300,0.082,0.412,0.506,0.000,7,0
4,84,354,3.91,67,0.245,0.000,0,0.00,0.000,0.284,0,0.187,0.000,0.284,0.000,0.187,0.407,0.407,5,0
5,85,354,3.91,67,0.260,0.000,0,0.00,0.000,0.382,0,0.155,0.027,0.176,0.026,0.243,0.365,0.365,7,0
6,86,354,3.91,67,0.333,0.000,0,0.00,0.000,0.333,0,0.000,0.000,0.333,0.207,0.198,0.297,0.297,5,0
7,87,354,3.91,67,0.297,0.108,0,0.00,0.000,0.297,0,0.000,0.000,0.297,0.000,0.263,0.369,0.369,5,0
8,88,354,3.91,67,0.211,0.000,0,0.00,0.220,0.220,0,0.000,0.220,0.129,0.000,1.000,0.000,0.000,5,0
9,89,354,3.91,67,0.250,0.000,0,0.25,0.000,0.250,0,0.000,0.000,0.250,0.000,0.267,0.367,0.367,5,0


In [19]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,237.729324,0.141756
1,Caution,A2,241.628937,0.253454
2,Caution,A3,233.251663,0.188435
3,Caution,A4,238.617310,0.199403
4,Caution,A5,245.000324,0.192250
5,Caution,A6,240.272316,0.259942
6,Good,B1,229.267177,0.215277
7,Good,B2,229.452561,0.183197
8,Good,B3,229.423476,0.235498
9,Good,B4,259.946172,0.235696


In [20]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,229.381071,0.099702,0.211324,0.026373,1
3,Formulation 4,260.016219,1.147150,0.266594,0.030378,1
4,Formulation 5,216.446263,6.555681,0.194452,0.013420,1
5,Formulation 6,245.330879,0.993630,0.220937,0.033685,1
6,Formulation 7,305.335526,18.141166,0.424973,0.038722,1
7,Formulation 8,234.042069,4.446378,0.215671,0.030945,1
8,Formulation 9,196.092131,8.431605,0.208728,0.026114,1
9,Formulation 10,306.996252,34.487581,0.434755,0.096278,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,80,354,3.91,67,0.300,0.000,0,0.00,0.000,0.300,...,0.382,0.382,5,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,81,354,3.91,67,0.359,0.000,0,0.00,0.000,0.359,...,0.403,0.403,5,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,82,354,3.91,67,0.233,0.000,0,0.00,0.000,0.319,...,0.564,0.000,6,0,Formulation 3,229.381071,0.099702,0.211324,0.026373,1
3,83,354,3.91,67,0.300,0.044,0,0.00,0.000,0.300,...,0.506,0.000,7,0,Formulation 4,260.016219,1.147150,0.266594,0.030378,1
4,84,354,3.91,67,0.245,0.000,0,0.00,0.000,0.284,...,0.407,0.407,5,0,Formulation 5,216.446263,6.555681,0.194452,0.013420,1
5,85,354,3.91,67,0.260,0.000,0,0.00,0.000,0.382,...,0.365,0.365,7,0,Formulation 6,245.330879,0.993630,0.220937,0.033685,1
6,86,354,3.91,67,0.333,0.000,0,0.00,0.000,0.333,...,0.297,0.297,5,0,Formulation 7,305.335526,18.141166,0.424973,0.038722,1
7,87,354,3.91,67,0.297,0.108,0,0.00,0.000,0.297,...,0.369,0.369,5,0,Formulation 8,234.042069,4.446378,0.215671,0.030945,1
8,88,354,3.91,67,0.211,0.000,0,0.00,0.220,0.220,...,0.000,0.000,5,0,Formulation 9,196.092131,8.431605,0.208728,0.026114,1
9,89,354,3.91,67,0.250,0.000,0,0.25,0.000,0.250,...,0.367,0.367,5,0,Formulation 10,306.996252,34.487581,0.434755,0.096278,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,80,354,3.91,67,0.300,0.000,0,0.00,0.000,0.300,...,5,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,81,354,3.91,67,0.359,0.000,0,0.00,0.000,0.359,...,5,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,82,354,3.91,67,0.233,0.000,0,0.00,0.000,0.319,...,6,0,Formulation 3,229.381071,0.099702,0.211324,0.026373,1,466.0,0
3,83,354,3.91,67,0.300,0.044,0,0.00,0.000,0.300,...,7,0,Formulation 4,260.016219,1.147150,0.266594,0.030378,1,600.0,0
4,84,354,3.91,67,0.245,0.000,0,0.00,0.000,0.284,...,5,0,Formulation 5,216.446263,6.555681,0.194452,0.013420,1,490.0,0
5,85,354,3.91,67,0.260,0.000,0,0.00,0.000,0.382,...,7,0,Formulation 6,245.330879,0.993630,0.220937,0.033685,1,520.0,0
6,86,354,3.91,67,0.333,0.000,0,0.00,0.000,0.333,...,5,0,Formulation 7,305.335526,18.141166,0.424973,0.038722,1,666.0,0
7,87,354,3.91,67,0.297,0.108,0,0.00,0.000,0.297,...,5,0,Formulation 8,234.042069,4.446378,0.215671,0.030945,1,594.0,0
8,88,354,3.91,67,0.211,0.000,0,0.00,0.220,0.220,...,5,0,Formulation 9,196.092131,8.431605,0.208728,0.026114,1,422.0,0
9,89,354,3.91,67,0.250,0.000,0,0.25,0.000,0.250,...,5,0,Formulation 10,306.996252,34.487581,0.434755,0.096278,1,500.0,0


In [23]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
2,1,0.229381,0.000100,0.233,0.0,0.211324,0.026373,0.500000
3,1,0.260016,0.001147,0.300,0.0,0.266594,0.030378,0.583333
4,1,0.216446,0.006556,0.245,0.0,0.194452,0.013420,0.416667
5,1,0.245331,0.000994,0.260,0.0,0.220937,0.033685,0.583333
6,1,0.305336,0.018141,0.333,0.0,0.424973,0.038722,0.416667
7,1,0.234042,0.004446,0.297,0.0,0.215671,0.030945,0.416667
8,1,0.196092,0.008432,0.211,0.0,0.208728,0.026114,0.416667
9,1,0.306996,0.034488,0.250,0.0,0.434755,0.096278,0.416667


In [24]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
2,1,0.229381,0.000100,0.233,0.0,0.211324,0.026373,0.500000
3,1,0.260016,0.001147,0.300,0.0,0.266594,0.030378,0.583333
4,1,0.216446,0.006556,0.245,0.0,0.194452,0.013420,0.416667
5,1,0.245331,0.000994,0.260,0.0,0.220937,0.033685,0.583333
6,1,0.305336,0.018141,0.333,0.0,0.424973,0.038722,0.416667
7,1,0.234042,0.004446,0.297,0.0,0.215671,0.030945,0.416667
8,1,0.196092,0.008432,0.211,0.0,0.208728,0.026114,0.416667
9,1,0.306996,0.034488,0.250,0.0,0.434755,0.096278,0.416667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,80,354,3.91,67,0.300,0.000,0,0.0,0.0,0.300,...,0.416667,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,81,354,3.91,67,0.359,0.000,0,0.0,0.0,0.359,...,0.416667,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,82,354,3.91,67,0.233,0.000,0,0.0,0.0,0.319,...,0.500000,0,Formulation 3,0.229381,0.000100,0.211324,0.026373,1,0.233,0
3,83,354,3.91,67,0.300,0.044,0,0.0,0.0,0.300,...,0.583333,0,Formulation 4,0.260016,0.001147,0.266594,0.030378,1,0.300,0
4,84,354,3.91,67,0.245,0.000,0,0.0,0.0,0.284,...,0.416667,0,Formulation 5,0.216446,0.006556,0.194452,0.013420,1,0.245,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-12 08:43:39] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 80 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.416667, 0)}.
[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 81 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.416667, 0)}.
[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 82 with data: {'Solu': (0.233, 0), 'Size': (0.229381, 0.0001), 'PDI': (0.211324, 0.026373), 'Complexity': (0.5, 0)}.
[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 83 with data: {'Solu': (0.3, 0), 'Size': (0.260016, 0.001147), 'PDI': (0.266594, 0.030378), 'Complexity': (0.583333, 0)}.
[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 84 with data: {'Solu': (0.245, 0), 'Size': (0.216446, 0.006556), 'PDI': (0.194452, 0.01342), 'Complexity': (0.416667, 0)}.
[INFO 02-12 08:43:40] ax.service.ax_client: Completed trial 85 with data: {'Solu': (0.26, 0), 'Size': (0.245331, 0.000994), 'PDI': (0.